<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Backpropagation Practice

## *Data Science Unit 4 Sprint 2 Assignment 2*

Implement a 3 input, 4 node hidden-layer, 1 output node Multilayer Perceptron on the following dataset:

| x1 | x2 | x3 | y |
|----|----|----|---|
| 0  | 0  | 1  | 0 |
| 0  | 1  | 1  | 1 |
| 1  | 0  | 1  | 1 |
| 0  | 1  | 0  | 1 |
| 1  | 0  | 0  | 1 |
| 1  | 1  | 1  | 0 |
| 0  | 0  | 0  | 0 |

If you look at the data you'll notice that the first two columns behave like an XOR gate while the last column is mostly just noise. Remember that creating an XOR gate was what the perceptron was criticized for not being able to learn. 

In [5]:
import numpy as np
import pandas as pd

np.random.seed(42)

X = np.array(([0,0,1],
              [0,1,1],
              [1,0,1],
              [0,1,0],
              [1,0,0],
              [1,1,1],
              [0,0,0]))

y = np.array(([0],
              [1],
              [1],
              [1],
              [1],
              [0],
              [0]))

In [14]:
class NeuralNetwork:
    
    def __init__(self):
        # Set up architecture
        self.inputs = 3
        self.hiddenNodes = 4
        self.outputNodes = 1
        
        # Initialize weights
        # 3 x 4
        # Input to Hidden (1st set of weights)
        self.weights1 = np.random.randn(self.inputs, self.hiddenNodes)
        # 4 x 1
        # Hidden to Output (2nd set of weights)
        self.weights2 = np.random.randn(self.hiddenNodes, self.outputNodes)
        
    def sigmoid(self, s):
        return 1 / (1 + np.exp(-s))
    
    def sigmoidPrime(self, s):
        ss = self.sigmoid(s)
        return ss * (1 - ss)
    
    def feed_forward(self, X):
        """
        Calculate the NN inference using feed forward.
        """
        
        # Weighted sum
        self.hidden_sum = np.dot(X, self.weights1)
        
        # Activate
        self.activated_hidden = self.sigmoid(self.hidden_sum)
        
        # Weighted sum of activated hidden (which output layer will use)
        self.output_sum = np.dot(self.activated_hidden, self.weights2)
        
        # Final activation of output (the prediction)
        self.activated_output = self.sigmoid(self.output_sum)
        
        return self.activated_output
    
    def backward(self, X, y, o):
        """
        Backpropagation through the network
        """
        
        self.o_error = y - o  # Error in the output
        
        # Apply the derivative of sigmoid to error
        self.o_delta = self.o_error * self.sigmoidPrime(o)
        
        # z2 error: how much were our output layer weights off
        self.z2_error = self.o_delta.dot(self.weights2.T)
        
        # z2 delta: how much were the weights off?
        self.z2_delta = self.z2_error*self.sigmoidPrime(self.activated_hidden)
        
        self.weights1 += X.T.dot(self.z2_delta)  # Adjust first set (input -> hidden) weights
        self.weights2 += self.activated_hidden.T.dot(self.o_delta)  # Adjust second set
        
    def train(self, X, y):
        o = self.feed_forward(X)
        self.backward(X, y, o)

In [17]:
# Train my 'net
nn = NeuralNetwork()

# Number of Epochs / Iterations
for i in range(10000):
    if (i+1 in [1,2,3,4,5]) or ((i+1) % 1000 ==0):
        print('+' + '---' * 3 + f'EPOCH {i+1}' + '---'*3 + '+')
        print('Input: \n', X)
        print('Actual Output: \n', y)
        print('Predicted Output: \n', str(nn.feed_forward(X)))
        print("Loss: \n", str(np.mean(np.square(y - nn.feed_forward(X)))))
    nn.train(X,y)

+---------EPOCH 1---------+
Input: 
 [[0 0 1]
 [0 1 1]
 [1 0 1]
 [0 1 0]
 [1 0 0]
 [1 1 1]
 [0 0 0]]
Actual Output: 
 [[0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]]
Predicted Output: 
 [[0.2389256 ]
 [0.31963415]
 [0.20994364]
 [0.39624124]
 [0.26912805]
 [0.26117096]
 [0.30982419]]
Loss: 
 0.3152959896205795
+---------EPOCH 2---------+
Input: 
 [[0 0 1]
 [0 1 1]
 [1 0 1]
 [0 1 0]
 [1 0 0]
 [1 1 1]
 [0 0 0]]
Actual Output: 
 [[0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]]
Predicted Output: 
 [[0.33255485]
 [0.41878625]
 [0.30346117]
 [0.48317791]
 [0.35881093]
 [0.35661401]
 [0.39577711]]
Loss: 
 0.2708014395002984
+---------EPOCH 3---------+
Input: 
 [[0 0 1]
 [0 1 1]
 [1 0 1]
 [0 1 0]
 [1 0 0]
 [1 1 1]
 [0 0 0]]
Actual Output: 
 [[0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]]
Predicted Output: 
 [[0.39677361]
 [0.48375345]
 [0.37088171]
 [0.53835782]
 [0.4212923 ]
 [0.42263164]
 [0.45281568]]
Loss: 
 0.25020075245853185
+---------EPOCH 4---------+
Input: 
 [[0 0 1]
 [0 1 1]
 [1 0 1]
 [0 1 0]
 [1 0 0]
 [1 1 1]
 [0 0 0]

## Try building/training a more complex MLP on a bigger dataset.

Use the [MNIST dataset](http://yann.lecun.com/exdb/mnist/) to build the cannonical handwriting digit recognizer and see what kind of accuracy you can achieve. 

If you need inspiration, the internet is chalk-full of tutorials, but I want you to see how far you can get on your own first. I've linked to the original MNIST dataset above but it will probably be easier to download data through a neural network library. If you reference outside resources make sure you understand every line of code that you're using from other sources, and share with your fellow students helpful resources that you find.


### Parts
1. Gathering & Transforming the Data
2. Making MNIST a Binary Problem
3. Estimating your Neural Network (the part you focus on)

### Gathering the Data 

`keras` has a handy method to pull the mnist dataset for you. You'll notice that each observation is a 28x28 arrary which represents an image. Although most Neural Network frameworks can handle higher dimensional data, that is more overhead than necessary for us. We need to flatten the image to one long row which will be 784 values (28X28). Basically, you will be appending each row to one another to make on really long row. 

In [1]:
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [2]:
# input image dimensions
img_rows, img_cols = 28, 28

In [14]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [15]:
x_train = x_train.reshape(x_train.shape[0], img_rows * img_cols)
x_test = x_test.reshape(x_test.shape[0], img_rows * img_cols)

# Normalize Our Data
x_train = x_train / 255
x_test = x_test / 255

x_train = x_train.astype('float16')
x_test = x_test.astype('float16')

In [16]:
# Now the data should be in a format you're more familiar with
x_train.shape

(60000, 784)

### Making MNIST a Binary Problem 
MNIST is multiclass classification problem; however we haven't covered all the necessary techniques to handle this yet. You would need to one-hot encode the target, use a different loss metric, and use softmax activations for the last layer. This is all stuff we'll cover later this week, but let us simply the problem for now: Zero or all else.

In [17]:
import numpy as np

y_temp = np.zeros(y_train.shape)
y_temp[np.where(y_train == 0.0)[0]] = 1
y_train = y_temp

y_temp = np.zeros(y_test.shape)
y_temp[np.where(y_test == 0.0)[0]] = 1
y_test = y_temp

In [18]:
# A Nice Binary target for ya to work with
y_train

array([0., 1., 0., ..., 0., 0., 0.])

### Estimating Your `net

In [59]:
##### Your Code Here #####

class NeuralNetwork:
    
    def __init__(self, learning_rate):
        self.learning_rate = learning_rate
        
        # Set up architecture
        self.inputs = 784
        self.hiddenNodes = 500
        self.outputNodes = 1
        
        # Initialize weights
        # 784 x 500
        # Input to Hidden (1st set of weights)
        self.weights1 = np.random.randn(self.inputs, self.hiddenNodes)
        self.weights1 = self.weights1 / self.weights1.shape[0]
        # 500 x 1
        # Hidden to Output (2nd set of weights)
        self.weights2 = np.random.randn(self.hiddenNodes, self.outputNodes)
        self.weights2 = self.weights2 / self.weights2.shape[0]
        
    def sigmoid(self, s):
        return 1 / (1 + np.exp(-s))
    
    def sigmoidPrime(self, s):
        ss = self.sigmoid(s)
        return ss * (1 - ss)
    
    def feed_forward(self, X):
        """
        Calculate the NN inference using feed forward.
        """
        
        # Weighted sum
        self.hidden_sum = np.dot(X, self.weights1)
        
        # Activate
        self.activated_hidden = self.sigmoid(self.hidden_sum)
        
        # Weighted sum of activated hidden (which output layer will use)
        self.output_sum = np.dot(self.activated_hidden, self.weights2)
        
        # Final activation of output (the prediction)
        self.activated_output = self.sigmoid(self.output_sum)
        
        return self.activated_output
    
    def backward(self, X, y, o):
        """
        Backpropagation through the network
        """
        
        self.o_error = y - o  # Error in the output
        
        # Apply the derivative of sigmoid to error
        self.o_delta = self.o_error * self.sigmoidPrime(o)
        
        # z2 error: how much were our output layer weights off
        self.z2_error = self.o_delta.dot(self.weights2.T)
        
        # z2 delta: how much were the weights off?
        self.z2_delta = self.z2_error*self.sigmoidPrime(self.activated_hidden)
        

        self.weights1 += self.learning_rate * np.reshape(X, (-1, 1)).dot(np.reshape(self.z2_delta, (-1, 1)).T)  # Adjust first set (input -> hidden) weights
        self.weights2 += self.learning_rate * np.reshape(self.activated_hidden, (-1, 1)).dot(np.reshape(self.o_delta, (-1, 1)).T)  # Adjust second set
        
    def train(self, X, y):
        o = self.feed_forward(X)
        self.backward(X, y, o)

In [79]:
# Train my 'net
learning_rate = 0.1
nn = NeuralNetwork(learning_rate)
epochs = 500
batch_size = 100

# Number of Epochs / Iterations
losses = []
for i in range(epochs):
    loss = 0
    for k in range(batch_size):
        col = np.random.randint(x_train.shape[0])
        input_vector = x_train[col, :]
        target_vector = y_train[col]
        nn.train(input_vector, target_vector)
        loss += np.linalg.norm(nn.sigmoid(np.dot(nn.weights2.T, nn.sigmoid(np.dot(nn.weights1.T,input_vector)))) - target_vector, 2)
    if (i+1) % 10 == 0:
        print('+' + '---' * 3 + f'EPOCH {i+1}' + '---'*3 + '+')
        print("Loss: \n", str(loss/batch_size))
    losses.append(loss)

+---------EPOCH 10---------+
Loss: 
 0.08109902882443372
+---------EPOCH 20---------+
Loss: 
 0.05984987217268701
+---------EPOCH 30---------+
Loss: 
 0.0323089427472321
+---------EPOCH 40---------+
Loss: 
 0.019590319157876613
+---------EPOCH 50---------+
Loss: 
 0.012397815350112015
+---------EPOCH 60---------+
Loss: 
 0.009726950658278038
+---------EPOCH 70---------+
Loss: 
 0.009176680765845418
+---------EPOCH 80---------+
Loss: 
 0.01116878544400676
+---------EPOCH 90---------+
Loss: 
 0.009402452695599462
+---------EPOCH 100---------+
Loss: 
 0.0077426612054147335
+---------EPOCH 110---------+
Loss: 
 0.0040677229780481505
+---------EPOCH 120---------+
Loss: 
 0.017187370934296876
+---------EPOCH 130---------+
Loss: 
 0.01890746121421914
+---------EPOCH 140---------+
Loss: 
 0.008281570857835043
+---------EPOCH 150---------+
Loss: 
 0.005703182534885402
+---------EPOCH 160---------+
Loss: 
 0.009196205185360972
+---------EPOCH 170---------+
Loss: 
 0.018974454545765063
+---------

## Stretch Goals: 

- Make MNIST a multiclass problem using cross entropy & soft-max
- Implement Cross Validation model evaluation on your MNIST implementation 
- Research different [Gradient Descent Based Optimizers](https://keras.io/optimizers/)
 - [Siraj Raval the evolution of gradient descent](https://www.youtube.com/watch?v=nhqo0u1a6fw)
- Build a housing price estimation model using a neural network. How does its accuracy compare with the regression models that we fit earlier on in class?